# Code for registering 2d FOVs using keypoint registration
1) TODO: based on this improve the CP target-selection script (the segmentation will be way better with motion correction)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import tifffile as tiff
from skimage.transform import warp, resize
import napari


from photostim_deve.image_analysis.plot import plot_motcorr_comparison, plot_segmentation_overlay_dict, plot_image_seg_xy_stim, plot_keypoints_scatter
from photostim_deve.image_analysis.io import get_all_fov_image, get_s2p_image, get_xy_stim, save_keypoints, load_keypoints, get_t2p_s2p_indices_session, get_s2p_rois_filt
from photostim_deve.image_analysis.segment import segment_fov_cpsam, get_cent_from_seg
from photostim_deve.image_analysis.register import register_keypoints_affine, match_ref_moving

In [ ]:
# set params
subject = 'jm065'
sat_perc = 99.99

fov_imsize = (1024, 1024)  # size of the FOV in pixels (assumed square)
s2p_imsize = (512, 512)  # size used for Suite2p processing

n_stim_cell = 45 # how many 'cell' points were stimulated
n_stim_ctrl = 5 # how many 'control' points were stimulated

session_reg_idx = 0  # index of the session used as reference for FOV registration across sessions (usually should be 0 for the first session)

force_recompute = False  # whether to force recomputation of motion correction and segmentation even if files already exist

# motion correction
run_motcorr = True
nimg_init = 128  # number of frames used to compute the reference image for motion correction

# choosing Suite2p ROIs to consider for matching
filt_by = 't2p'  # filter Suite2p ROIs by 't2p' (track2p matching) or 'iscell_cell_prob' or 'iscell_manual_cur'
track2p_dirname='track2p_a' # this applies only if above filt_by is 't2p' (name of the track2p directory to get the matching indices from)
cell_prob_thr = 0.5 # this applies only if above filt_by is 'iscell_cell_prob' (threshold for cell probability to consider an ROI as a cell based on s2p classifier)

# parameters used for matching
max_dist_px = 7 # hard threshold on distance in pixels for matching points between two sets of coordinates

# session type (either '_a' - spontaneous or '_s' - evoked) 
session_type = '_a' # TODO: PAY ATTENTION FOR NOW THAT IT MATCHES THE track2p_dirname!!!

In [ ]:
subject_path = os.path.join('data_proc', 'jm', subject)
all_session_path = sorted([os.path.join(subject_path, d) for d in os.listdir(subject_path) if os.path.isdir(os.path.join(subject_path, d)) and session_type in d])
session_path = all_session_path[session_reg_idx]

match_save_dir = os.path.join(session_path, 'match_s2p_fov_stim')

keypoints_save_path = os.path.join(session_path, 'fov_reg_keypoints.csv')


fov_s2p_px_fact = fov_imsize[0] / s2p_imsize[0] # both have the same aspect ratio

In [ ]:
all_fov_image = get_all_fov_image(subject_path, 
                                  session_type=session_type,
                                  session_reg_idx = session_reg_idx, 
                                  run_motcorr=run_motcorr, 
                                  fov_imsize=fov_imsize, 
                                  nimg_init=nimg_init,
                                  force_recompute=force_recompute)

In [ ]:
plot_motcorr_comparison(all_fov_image, sat_perc=sat_perc, crop=(64, 64))

In [ ]:
all_fov_image_seg = segment_fov_cpsam(all_fov_image, flow_threshold=0.4, cellprob_threshold=0.0, force_recompute=force_recompute, save_path=session_path)

In [ ]:
plot_segmentation_overlay_dict(all_fov_image_seg, sat_perc=sat_perc)

In [ ]:
s2p_image = get_s2p_image(session_path)

In [ ]:
x_stim, y_stim = get_xy_stim(session_path, session_type=session_type)

In [ ]:
plot_image_seg_xy_stim(all_fov_image['1100nm'], x_stim=x_stim, y_stim=y_stim, segmentation=all_fov_image_seg['1100nm_seg'], sat_perc=sat_perc, fov_s2p_px_fact=fov_s2p_px_fact)


In [ ]:
s2p_image_upscaled = resize(s2p_image, fov_imsize, preserve_range=True, anti_aliasing=True).astype(s2p_image.dtype)

In [ ]:
keypoints_save_path

In [ ]:
if not os.path.exists(keypoints_save_path) or force_recompute:
    import napari

    viewer = napari.Viewer()

    viewer.add_image(s2p_image_upscaled, name='s2p_mean_image', colormap='green', contrast_limits=(np.percentile(s2p_image_upscaled, 0.1), np.percentile(s2p_image_upscaled, sat_perc)))
    viewer.add_points(name='s2p_keypoints', size=5, face_color='green')
    viewer.add_image(all_fov_image['1100nm'], name='fov_1100nm', colormap='magenta', contrast_limits=(np.percentile(all_fov_image['1100nm'], 0.1), np.percentile(all_fov_image['1100nm'], sat_perc)))
    viewer.add_points(name='fov_1100nm_keypoints', size=5, face_color='magenta')
    napari.run()
    


In [ ]:
if not os.path.exists(keypoints_save_path) or force_recompute:
    save_keypoints(viewer, keypoints_save_path)

In [ ]:
x_kp_s2p, y_kp_s2p, x_kp_fov, y_kp_fov = load_keypoints(keypoints_save_path)

In [ ]:
x_kp_fov_reg, y_kp_fov_reg, transform = register_keypoints_affine(x_kp_s2p, y_kp_s2p, x_kp_fov, y_kp_fov)
plot_keypoints_scatter(x_kp_s2p, y_kp_s2p, x_kp_fov, y_kp_fov, x_kp_fov_reg, y_kp_fov_reg)

In [ ]:
x_stim_upscaled, y_stim_upscaled = x_stim * fov_s2p_px_fact, y_stim * fov_s2p_px_fact
x_stim_upscaled_reg, y_stim_upscaled_reg = transform.inverse(np.stack([x_stim_upscaled, y_stim_upscaled], axis=1)).T


In [ ]:
# now get and transform the centroids of the 1100nm CPs
x_fov, y_fov = get_cent_from_seg(all_fov_image_seg['1100nm_seg'])
x_fov_reg, y_fov_reg = transform.inverse(np.stack([x_fov, y_fov], axis=1)).T

In [ ]:
# now apply transform to the image
fov_image = all_fov_image['1100nm']
fov_image_reg = warp(fov_image, inverse_map=transform.inverse, output_shape=fov_imsize)

fov_seg = all_fov_image_seg['1100nm_seg']
fov_seg_reg = warp(fov_seg, inverse_map=transform.inverse, output_shape=fov_imsize, order=0, preserve_range=True).astype(fov_seg.dtype)


In [ ]:
s2p_image_upscaled = resize(s2p_image, fov_imsize, preserve_range=True, anti_aliasing=True).astype(s2p_image.dtype)

In [ ]:
t2p_idxs_session = get_t2p_s2p_indices_session(subject_path, track2p_dirname=track2p_dirname, session_reg_idx=session_reg_idx)

In [ ]:
roi_s2p, x_s2p_med, y_s2p_med, idxs_filt = get_s2p_rois_filt(session_path, filt_by='t2p', t2p_idxs_session=t2p_idxs_session)

In [ ]:
x_s2p_med_upscaled = x_s2p_med * fov_s2p_px_fact
y_s2p_med_upscaled = y_s2p_med * fov_s2p_px_fact

In [ ]:
row_ind_s2p_fov, col_ind_s2p_fov = match_ref_moving(x_s2p_med_upscaled, y_s2p_med_upscaled, x_fov_reg, y_fov_reg, max_dist_px=max_dist_px)
row_ind_s2p_stim, col_ind_s2p_stim = match_ref_moving(x_s2p_med_upscaled, y_s2p_med_upscaled, x_stim_upscaled_reg, y_stim_upscaled_reg, max_dist_px=max_dist_px)

In [ ]:
viewer = napari.Viewer()

viewer.add_image(fov_image_reg, name='fov_1100nm_registered', colormap='magenta', blending='additive', contrast_limits=(np.percentile(fov_image_reg, 0.1), np.percentile(fov_image_reg, 99.9)))
viewer.add_image(s2p_image_upscaled, name='s2p_mean_image_registered', colormap='green', blending='additive', contrast_limits=(np.percentile(s2p_image_upscaled, 0.1), np.percentile(s2p_image_upscaled, sat_perc)))
viewer.add_image(fov_seg_reg>0, name='fov_1100nm_seg_registered', opacity=0.3, colormap='magenta')
viewer.add_points(np.stack([x_s2p_med_upscaled, y_s2p_med_upscaled], axis=1), name='s2p_rois_medians_upscaled', size=5, face_color='green')
viewer.add_points(np.stack([x_stim_upscaled_reg, y_stim_upscaled_reg], axis=1), name='stim_points_registered', symbol='x', size=5, face_color='cyan')
viewer.add_points(np.stack([x_fov_reg[col_ind_s2p_fov], y_fov_reg[col_ind_s2p_fov]], axis=1), name='matched_centroids_1100nm', size=15, border_color='yellow', border_width=0.2, face_color=[0,0,0,0], opacity=0.5)
viewer.add_points(np.stack([x_stim_upscaled_reg[col_ind_s2p_stim], y_stim_upscaled_reg[col_ind_s2p_stim]], axis=1), name='matched_stim_points', symbol='s', size=20, border_color='white', border_width=0.2, face_color=[0,0,0,0], opacity=0.5)
napari.run()
    

In [ ]:
# calculate proportions
n_stim_to_s2p = len(col_ind_s2p_stim)
n_fov_to_s2p = len(col_ind_s2p_fov)
n_s2p = len(x_s2p_med)

prop_stim_to_s2p = n_stim_to_s2p / n_stim_cell
prop_fov_to_s2p = n_fov_to_s2p / n_s2p

print(f'Identified {n_fov_to_s2p} (/ {n_s2p}) tracked cells as expressing opsin, corresponding to: {prop_fov_to_s2p:.3f}.')
print(f'Tracked {n_stim_to_s2p} (/ {n_stim_cell}) stimulated cells, corresponding to: {prop_stim_to_s2p:.3f}.')

In [ ]:
# TODO: export this in a format that will be easy to match with longipy
# make a numpy array with indexes that are True for stimulated & tracked cells
is_stim_and_s2p = row_ind_s2p_stim
is_stim_and_s2p_idx = col_ind_s2p_stim # the index of that ROI according to the order of stimulation (from Bruker MarkPoints)
is_fov_and_s2p = row_ind_s2p_fov
is_fov_and_s2p_idx = col_ind_s2p_fov # the index of that ROI in the CP segmentation (for now not really needed)

# Save these in the a separate folder
if not os.path.exists(match_save_dir):
    os.makedirs(match_save_dir)
else:
    print(f'Matching save directory {match_save_dir} already exists.')

np.save(os.path.join(match_save_dir, 'is_stim_and_s2p.npy'), is_stim_and_s2p)
np.save(os.path.join(match_save_dir, 'is_stim_and_s2p_idx.npy'), is_stim_and_s2p_idx)
np.save(os.path.join(match_save_dir, 'is_fov_and_s2p.npy'), is_fov_and_s2p)
np.save(os.path.join(match_save_dir, 'is_fov_and_s2p_idx.npy'), is_fov_and_s2p_idx)